In [3]:
!pip install transformers
!pip install datasets
!pip install sacremoses
!pip install peft
!pip install bert-score

  Using cached peft-0.10.0-py3-none-any.whl (199 kB)
  Using cached accelerate-0.29.3-py3-none-any.whl (297 kB)


In [4]:
import pandas as pd
import numpy as np
import os
df=pd.read_excel('/content/DB.xlsx')
df.head(5)

,egyption_Text,english_Text,category,sub_category
0,"يا ""سندباد""، اعتبر الموضوع دا خلصان.","Sinbad, consider this matter resolved.",Subtitles,ألف ليلة وليلة الدحيح
1,انت بس ابعتلي الولا بالورق،\nوأنا هأخلّصلك كل ...,"Just send me the paper, and\nI will finish eve...",Subtitles,ألف ليلة وليلة الدحيح
2,ماشي يا حبيبي. في رعاية الله.,OK dear. God protect you.,Subtitles,ألف ليلة وليلة الدحيح
3,"سلامي لـ""الشاطر (حسن)""، هه!يلّا يا حبيبي، في ر...","Greetings to Alaadin!Goodbye now.Good morning,...",Subtitles,ألف ليلة وليلة الدحيح
4,والله صباح زي أي صباح!اتفضل يا سيدي، اؤمر!عندي...,It's a morning\nlike any other morning.-Tell m...,Subtitles,ألف ليلة وليلة الدحيح


In [5]:
df.columns

Index(['egyption_Text', 'english_Text', 'category', 'sub_category'], dtype='object')

In [6]:
df.drop(['category', 'sub_category'], axis=1, inplace=True)
df.columns

Index(['egyption_Text', 'english_Text'], dtype='object')

In [7]:
df.shape

(8744, 2)

In [8]:
max(df["egyption_Text"].apply(len))

28299

In [9]:
max(df["english_Text"].apply(len))

32767

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
checkpoint='facebook/m2m100_418M'
tokenizer=AutoTokenizer.from_pretrained(checkpoint)
model=AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [11]:
from sklearn.model_selection import train_test_split


# Split the data into train and temp sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Optionally, you might want to reset the indices
train = train_df.reset_index(drop=True)
val = val_df.reset_index(drop=True)

print(f'train shape{train.shape}')
print(f'val shape{val.shape}')

train shape(6995, 2)
val shape(1749, 2)


In [12]:
from datasets import load_dataset, Dataset, DatasetDict
train_dataset=Dataset.from_pandas(train)
val_dataset=Dataset.from_pandas(val)

In [13]:
data_dict=DatasetDict(
    {
        'train':train_dataset,
        'val':val_dataset
    }
)

In [14]:
from transformers import pipeline
# translation here is the task
pipe = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="en", tgt_lang="ar")

In [15]:
for idx, (english_text, arabic_text) in enumerate(zip(df['english_Text'], df['egyption_Text'])):
    if idx == 10:
        break
    print(f"{idx+1}. English : {english_text}  \n\nEgyptian: {arabic_text}")
    print("*"*10)


1. English : Sinbad, consider this matter resolved.  

Egyptian: يا "سندباد"، اعتبر الموضوع دا خلصان.
**********
2. English : Just send me the paper, and
I will finish everything for you.Okay?  

Egyptian: انت بس ابعتلي الولا بالورق،
وأنا هأخلّصلك كل حاجة.تمام؟
**********
3. English : OK dear. God protect you.  

Egyptian: ماشي يا حبيبي. في رعاية الله.
**********
4. English : Greetings to Alaadin!Goodbye now.Good morning, Mr. Genie!  

Egyptian: سلامي لـ"الشاطر (حسن)"، هه!يلّا يا حبيبي، في رعاية الله، في رعاية الله!صباح الخير يا أستاذ جنّي!
**********
5. English : It's a morning
like any other morning.-Tell me your command.
-I have a wish.  

Egyptian: والله صباح زي أي صباح!اتفضل يا سيدي، اؤمر!عندي أمنية.
**********
6. English : Good, keep it warm and safe.  

Egyptian: طب حلو،
ربنا يخلّيهالك وتشوفها عروسة، إن شاء الله!
**********
7. English : No, I'm here to apply for a wish.  

Egyptian: لأ، أنا جاي أقدّم على أمنية.
**********
8. English : What type of wish is it?  

Egyptian: آه! نو

In [16]:
for idx, i in enumerate(df['english_Text']):
    if idx == 10:
        break
    print(f"{idx+1}. English : {i} \n\nTranslation to Egyptian without fine tunning: {pipe(i)[0]['translation_text']}")
    print("*"*10)

1. English : Sinbad, consider this matter resolved. 

Translation to Egyptian without fine tunning: بوتفليقة، انظر إلى هذه المسألة.
**********
2. English : Just send me the paper, and
I will finish everything for you.Okay? 

Translation to Egyptian without fine tunning: فقط أرسل لي الورق، وسأكمل كل شيء من أجلك.حسنا؟
**********
3. English : OK dear. God protect you. 

Translation to Egyptian without fine tunning: يا حبيبتي الله يحميك
**********
4. English : Greetings to Alaadin!Goodbye now.Good morning, Mr. Genie! 

Translation to Egyptian without fine tunning: السلام عليكم ورحمة الله وبركاته السلام عليكم ورحمة الله وبركاته السلام عليكم ورحمة الله وبركاته
**********
5. English : It's a morning
like any other morning.-Tell me your command.
-I have a wish. 

Translation to Egyptian without fine tunning: إنها صباح مثل أي صباح آخر - أخبرني بأمرك - لدي رغبة.
**********
6. English : Good, keep it warm and safe. 

Translation to Egyptian without fine tunning: حسنا ، احفظها دافئة وآمنة.
*******

In [17]:
tokenizer = AutoTokenizer.from_pretrained("facebook/m2m100_418M", tgt_lang="ar")

ar = data_dict['train']['egyption_Text'][0]
en = data_dict['train']['english_Text'][0]
inputs = tokenizer(en)
targets = tokenizer(text_target = ar)

In [18]:
def tokenizer_func(batch): #Just take batch of data not the full data
    inputs=tokenizer([x for x in batch['english_Text']],
                    truncation=True,
                    max_length=160)

    targets=tokenizer([x for x in batch['egyption_Text']],
                     truncation=True,
                     max_length=160)

    inputs['labels']=targets['input_ids']
    return inputs

In [19]:
tokenized_data=data_dict.map(
    tokenizer_func,
    batched=True,
    remove_columns = data_dict['train'].column_names # Remove columns that is not relevant to tokenizer
)

Map:   0%|          | 0/6995 [00:00<?, ? examples/s]

Map:   0%|          | 0/1749 [00:00<?, ? examples/s]

In [20]:
from transformers import DataCollatorForSeq2Seq
data_collator=DataCollatorForSeq2Seq(
    tokenizer,
    model=model
)

In [21]:
from datasets import load_metric

bert_metric = load_metric('bertscore')

def compute_metrics(preds_and_labels):
    preds, labels = preds_and_labels

    '''
    decodes the model predictions using the tokenizer.
    It converts the token IDs in the predictions to human-readable text,
    skipping special tokens such as padding or separator tokens.
    '''
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    '''
    The first argument (labels != -100) is a boolean condition that checks if each element in the labels array is not equal to -100. This creates a boolean mask where True indicates positions where the condition is satisfied and False otherwise.
The second argument (labels) is the value to be returned for positions where the condition is True. In this case, it's the original value in the labels array.
The third argument (tokenizer.pad_token_id) is the value to be returned for positions where the condition is False. It represents the token ID for padding obtained from the tokenizer.
    '''
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = bert_metric.compute(predictions=decoded_preds, references=decoded_labels, lang='fr')

    #This line returns the computed BERTScore as the evaluation metric. It computes the mean F1 score
    return {'BERT Score': np.mean(result['f1'])}

<ipython-input-21-cc6db156b1da>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bert_metric = load_metric('bertscore')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [22]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
training_args = Seq2SeqTrainingArguments(
    output_dir='finetuned2',#Directory where model checkpoints and other outputs will be saved.
    per_device_train_batch_size=2,# Batch size per GPU/CPU for training
    per_device_eval_batch_size=2,# Batch size per GPU/CPU for evaluation
    predict_with_generate=True,#generate predictions during evaluation

    evaluation_strategy='epoch',# Strategy for evaluation during training (e.g., 'epoch', 'steps').
    weight_decay=0.01, #Weight decay rate for regularization.
    save_total_limit=1,#Maximum number of checkpoints to keep.
    learning_rate=1e-4,
    num_train_epochs=5
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['val'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bert score
1,2.248200,1.963347,0.807193
2,1.443200,1.728197,0.830981
3,0.835600,1.648211,0.838262


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instea

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instea

Epoch,Training Loss,Validation Loss,Bert score
1,2.248200,1.963347,0.807193
2,1.443200,1.728197,0.830981
3,0.835600,1.648211,0.838262
4,0.414300,1.656505,0.846052
5,0.167000,1.678446,0.850337


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instea

TrainOutput(global_step=17490, training_loss=1.063779156762849, metrics={'train_runtime': 10958.7779, 'train_samples_per_second': 3.192, 'train_steps_per_second': 1.596, 'total_flos': 1831385391710208.0, 'train_loss': 1.063779156762849, 'epoch': 5.0})

In [24]:
trainer.save_model('M2M_EN-TO-AR')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}


In [25]:
import shutil
shutil.make_archive("egy-eng", 'zip', "/content/M2M_EN-TO-AR")

'/content/egy-eng.zip'

In [27]:
translator = pipeline('translation', model='M2M_EN-TO-AR', device=0)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
data_dict['val']['english_Text'][0]

In [ ]:
pipe = pipeline('translation', model=checkpoint, src_lang='en', tgt_lang='ar')
idx = 0
for i, j in zip(df['english_Text'], df['egyption_Text']):
    if idx == 10:
        break
    print(f"English: {i}\n\nPrediction without fine tuning: {pipe(i, src_lang='en', tgt_lang='ar')[0]['translation_text']}\n\nPrediction with fine tuning: {translator(i, src_lang='en', tgt_lang='ar')[0]['translation_text']}\n\nActual Translation: {j}")
    print("*"*10)
    idx+=1

English: Sinbad, consider this matter resolved.

Prediction without fine tuning: بوتفليقة، انظر إلى هذه المسألة.

Prediction with fine tuning: "السندباد" هيعتبر المسألة دي حلّت،

Actual Translation: يا "سندباد"، اعتبر الموضوع دا خلصان.
**********
English: Just send me the paper, and
I will finish everything for you.Okay?

Prediction without fine tuning: فقط أرسل لي الورق، وسأكمل كل شيء من أجلك.حسنا؟

Prediction with fine tuning: انت بس ابعتلي الولا بالورق، وأنا هأخلّصلك كل حاجة.تمام؟

Actual Translation: انت بس ابعتلي الولا بالورق،
وأنا هأخلّصلك كل حاجة.تمام؟
**********
English: OK dear. God protect you.

Prediction without fine tuning: يا حبيبتي الله يحميك

Prediction with fine tuning: حبيبيبي. في رعاية الله.

Actual Translation: ماشي يا حبيبي. في رعاية الله.
**********
English: Greetings to Alaadin!Goodbye now.Good morning, Mr. Genie!

Prediction without fine tuning: السلام عليكم ورحمة الله وبركاته السلام عليكم ورحمة الله وبركاته السلام عليكم ورحمة الله وبركاته

Prediction with fine 